In [1]:
# ResNet：深度残差网络
# http://zh.gluon.ai/chapter_convolutional-neural-networks/resnet-gluon.html

from mxnet.gluon import nn
from mxnet import nd
import utils
from resnets import *

/home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 输入输出通道相同
blk = ResidualIdentity(3)
blk.initialize()

x = nd.random.uniform(shape=(4,3,96,96))
y = blk(x)
print('y.shape:', y.shape)
print(blk)

y.shape: (4, 3, 96, 96)
ResidualIdentity(
  (conv1): Conv2D(3 -> 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=3)
  (conv2): Conv2D(3 -> 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=3)
)


In [3]:
# 输入输出通道不同
blk2 = ResidualIdentity(8, same_shape=False)
blk2.initialize()
# print(blk2)
y2 = blk2(x)
print('y2.shape:',y2.shape)
print(blk2)

y2.shape: (4, 8, 48, 48)
ResidualIdentity(
  (conv1): Conv2D(3 -> 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=3)
  (conv2): Conv2D(8 -> 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=8)
  (conv3): Conv2D(3 -> 8, kernel_size=(1, 1), stride=(2, 2))
)


In [4]:
net = ResNet34(10, verbose=True)
net.initialize()

x = nd.random.uniform(shape=(4,3,96, 96))
y = net(x)
print(net)

Block 1 output: (4, 64, 45, 45)
Block 2 output: (4, 64, 22, 22)
Block 3 output: (4, 128, 11, 11)
Block 4 output: (4, 256, 6, 6)
Block 5 output: (4, 512, 3, 3)
Block 6 output: (4, 10)
ResNet34(
  (net): Sequential(
    (0): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2))
    (1): Sequential(
      (0): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(0, 0), ceil_mode=False)
      (1): ResidualIdentity(
        (conv1): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (conv2): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
      )
      (2): ResidualIdentity(
        (conv1): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=Fals

In [5]:
# 读取数据
import sys
sys.path.append('..')
import utils
from mxnet import autograd
from mxnet import gluon
from mxnet import nd
from mxnet import image
from mxnet import init

batch_size = 128
resize=224
learning_rate = 0.01

train_data, test_data = utils.load_data(batch_size=batch_size, resize=resize)

ctx = utils.try_gpu(0)
net = ResNet34(10, verbose=False)
net.initialize(ctx=ctx, init=init.Xavier())

In [6]:
utils.train(net=net, train_data=train_data, test_data=test_data,ctx=ctx,
            batch_size=batch_size, learning_rate=learning_rate)

Epoch 0. Loss: 1.3130, Train acc 0.7632, Test acc 0.8511, Time 343 sec
Epoch 1. Loss: 0.3616, Train acc 0.8675, Test acc 0.8833, Time 344 sec
Epoch 2. Loss: 0.2856, Train acc 0.8956, Test acc 0.9003, Time 333 sec
Epoch 3. Loss: 0.2396, Train acc 0.9133, Test acc 0.9004, Time 326 sec
Epoch 4. Loss: 0.2022, Train acc 0.9282, Test acc 0.9057, Time 328 sec
Epoch 5. Loss: 0.1700, Train acc 0.9409, Test acc 0.9126, Time 333 sec
Epoch 6. Loss: 0.1442, Train acc 0.9507, Test acc 0.9147, Time 326 sec
Epoch 7. Loss: 0.1172, Train acc 0.9616, Test acc 0.9147, Time 324 sec
Epoch 8. Loss: 0.0937, Train acc 0.9721, Test acc 0.9167, Time 316 sec
Epoch 9. Loss: 0.0708, Train acc 0.9803, Test acc 0.9182, Time 316 sec
